In [1]:
import csv
import string 

translator = str.maketrans('','',string.punctuation)

word_count = {}

text=open('sd.txt').read()

words = text.split()

for word in words:
    word = word.translate(translator).lower()
    count = word_count.get(word,0)
    count +=1
    word_count[word] =count
    
word_count_liste =sorted(word_count, key=word_count.get, reverse=True)

for word in word_count_liste[:10]:
    print(word,word_count[word])
    
output_file = open('words_count.csv','a')
writer = csv.writer(output_file)
writer.writerow(['word','count'])

for word in word_count_liste:
    writer.writerow([word,word_count[word]])
    
    


de 6
par 4
a 3
été 3
l’ecole 2
supérieure 2
d’informatique 2
esi 2
créée 2
en 2


In [2]:
import findspark
findspark.init('/opt/spark-3.0.1-bin-hadoop3.2') # Localisation de Apache Spark en local

In [3]:
import sys

from pyspark import SparkConf, SparkContext

sc = SparkContext("local","Exercice sur le Words count") # Création d'un context spark


In [12]:
words = sc.textFile("sd.txt").flatMap(lambda line: line.split(" "))

wordscounts = words.map(lambda word: (word,1)).reduceByKey(lambda key,values:key +values)

wordscounts.saveAsTextFile("/home/jose-arthur/Bureau/SD/spark/output/test")
    

In [ ]:
RECUPERER DES DONNEES SUR TWITTER OU SUR FACEBOOK A TRAVERS UN PROGRAMME PARALELLE EN PYTHON

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Feb 10 14:04:08 2021

@author: jose-arthur
"""

import threading
import time
import tweepy # Twitter Api to gattering data with python
import facebook  # Facebook Api to gattering data
import json
import codecs
import os

####input your credentials here
""" 
    The following shows the use of the token 
    for data collection on the social network Twitter!
"""
consumer_key = 'Y7eOVbjYWlAP4k6FZKbnDrvRH'
consumer_secret = 'kaae85mzyjXCxrf0hcMgC4aTerBJrHPVYR6wQvf9ZwuljsDfWH'
access_token = '1283183414221377537-GmXIyFKdhElR6FptUKNQAMerRtp3Ok'
access_token_secret = 'F6xBi9soNq0PCno2tKIDgNTZkOqelsSARJgx6crMqORZL'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)



"""

    The use of token for data collection on the social network Facebook!

"""

access_token_facebook = 'EAAEMoEaTd3wBAG6zbcby94XzlhCVdJE9bW0FK2kVOdxkWCZASAfrsPAOjplHqQZBNrDX77uAjvSX6Ek8A66dQJEBP7GVgaoVfkNGOPcUVdi9PMX9ZCKrfQeeB6OoofrokcU76yGnXmgT64B98OmNwZB4CcH8QkrcgfRp5O9jnZARTWmYO98gZAY4Ar83wb1sdkl3Ir2lCrW7lU4IfrjXugGQHr2ZBZBKRSf9LOwKGFNnuQZDZD'

start_time = time.time()

def task1():
    #First bloc to collect data for terrorism
    start_time_t1 = time.time()
    results = tweepy.Cursor(api.search,q="#Whuan",count=70,
                               lang="en",
                               since="2021-02-10").items()
    end_time_t1 = time.time()
    print("Le temps d'execution du task1{}", end_time_t1 -start_time_t1)
    print()
    fichier = open("/SD/output/para_whuan_data.csv", "a")
    for _result in results:
        print("Data collecting of Wuhan")
        fichier.write(str(_result.text).encoding('utf8'))
    fichier.close()

def task2():
    #Second bloc to collect data for covid19
    start_time_t2 = time.time()
    covids = tweepy.Cursor(api.search,q="#Coronavirus",count=70,
                               lang="en",
                               since="2021-02-10").items()
    end_time_t2 = time.time()
    print("Le temps d'execution du task2 {}", end_time_t2 -start_time_t2)
    print()
    
    fichier = open("/SD/output/para_coronavirus_data.csv", "a")
    for _result in covids:
        print("Data getting of Coronavirus")  
        fichier.write(str(_result.text).encoding('utf8'))
    fichier.close()

def task3():
    #Third bloc to collect data for covid19
    
    token = access_token_facebook
    graph = facebook.GraphAPI(token)   
    start_time_t3 = time.time()
    
    users_posts = graph.get_object('me',fields='posts.fields(id,name,gender,email).limit(2)')
    end_time_t3 = time.time()
    
    print("Le temps d'execution du task3-Facebook_Data {}", end_time_t3 -start_time_t3)
    print()
    
    fichier = codecs.open("/SD/para_facebook_data.json", "a", encoding='utf8')
    print("Data gattering of Graph API")
    fichier.write(str(json.dumps(users_posts, indent=4)))
    fichier.close()
    
end_time = time.time()
print ("Le temps d'execution global des trois Thread =", end_time -start_time)
        
if __name__ == "__main__":
 
    # print ID of current process
    print("ID of process running main program: {}".format(os.getpid()))
 
    # print name of main thread
    print("Main thread name: {}".format(threading.main_thread().name))
   
    # creating threads
   
    t1 = threading.Thread(target=task1, name='t1')
    t2 = threading.Thread(target=task2, name='t2')  
    t3 = threading.Thread(target=task3, name='t3')  
 
    # starting threads
    print("starting threads Now !")
    t1.start()
   
    t2.start()
    
    t3.start()
    
    # wait until all threads finish
    print("Wait until all threads finish")
    t1.join()
    
    t2.join()
    
    t3.join()
   

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql import SparkSession  #  pyspark pour la connexion entre spark et mongodb

In [ ]:
input_uri="mongodb://127.0.0.1/sd2.villes"
output_uri="mongodb://127.0.0.1/sd2.villes"

myspark = SparkSession \
    .builder \
    .appName("myGIS") \
    .config("spark.mongodb.input.uri", input_uri) \
    .config("spark.mongodb.output.uri", output_uri) \
    .config("spark.jars.packages","org.mongodb.spark:mongo-spark-connector_2.12:2.4.2") \
    .getOrCreate()

In [ ]:
df = myspark.read.format("com.mongodb.spark.sql.DefaultSource").load()   

print(df.printSchema()) 

df.show()

In [ ]:
schema = StructType([
    StructField("city", StringType()),
    StructField("state",  StringType()),
    StructField("pop",  IntegerType())
])

df = myspark.createDataFrame([('Arizona','AZ',15000),('Indian','IN',25000) ], schema)

df.show()

df.write.format('com.mongodb.spark.sql.DefaultSource').mode("append").save()

In [ ]:
import pymongo # Interface from Python <---> MongoDB
import pandas as pd
import threading
import time
import os
from pymongo import MongoClient
from bson.code import Code # Help MongoDB with map-reduce

client = MongoClient('mongodb://localhost:27017/')

db = client["sd2"]

collection = db["villes"]

mes_villes = collection.find()

# print(mes_villes.next())

# Map function 

mapper = Code("function(){emit(this.city,this.pop)}")


# Reduce function

reducer = Code("function (key,values) {return Array.sum(values)}")

# Result of query
start_time_t1 = time.time()
map_red = collection.map_reduce(mapper, reducer,'pop_count')
end_time_t1 = time.time()
print("Temps d'execution du task{} :", end_time_t1 -start_time_t1)


# Getting result

data = []

for result in map_red.find() :
    data.append(result)

data[:4]